## Cleaning

First, let's load and mop up the data a little bit. We want to drop anything where the headline is `NaN`. Fortunately, there aren't too many of those.

Secondly, a lot of the headline categories have many sub-categories. We end up with a ridiculously long list of category labels this way. Each of these likely only has so many headlines, and it's far more detailed than I'm concerned with here anyhow. I'll simplify those to be just the top-level categories.

In [1]:
import pandas as pd

df = pd.read_csv('data/irishtimes-date-text.csv')
df.head()

,publish_date,headline_category,headline_text
0,19960102,business,Smurfit's share price in retreat despite recor...
1,19960102,business,Jamont plans £5m investment to update plant
2,19960102,business,Management is blamed for most company failures
3,19960102,business,Forte expected to announce a special dividend ...
4,19960102,business,Accountancy firm adopts name change


In [2]:
# Remove blank headlines
df.dropna(axis='index', how='any', inplace=True)

# Simplify the category names
df['simple_category'] = df.headline_category.str.split('.').str.get(0)

In [3]:
from textblob import TextBlob

# Let's add some sentiment features to the data
headline_list = [TextBlob(headline).sentiment for headline in df['headline_text'].tolist()]
polarities = [sent.polarity for sent in headline_list]
subjectivities = [sent.subjectivity for sent in headline_list]

df['polarity'] = polarities
df['subjectivity'] = subjectivities

## Exploration
With some cleaning now out of the way, let's do a wee bit of exploration to see what we've got here.

## Prepping the Model

We'll create a small pipeline here with a bag-of-words model that we can categorize with.

We'll get a list of the unique category names and then add the labels. Then we'll create a variable with the headline text for predicting and a `dict` of labels, only of which will be `True` for a given headline.

In [4]:
import spacy

# Create the model
nlp = spacy.blank('en')

categorizer = nlp.create_pipe('textcat', config={'exclusive_classes': True,
                                                 # Try the ensemble again later
                                                 'architecture': 'ensemble',
                                                 'ngram_size': 3})
nlp.add_pipe(categorizer)

# Add the labels
cat_names = df['simple_category'].unique()

for cat in cat_names:
    categorizer.add_label(cat)

In [5]:
# from multiprocessing import Pool
from nltk.corpus import stopwords

# Tokenize and remove stopwords
stops = stopwords.words('english')

def clean_headlines(doc):
    return ' '.join([x.text.lower() for x in doc 
                     if x.text.lower() not in stops and not x.is_punct])

# Add some data cleaning before categorization.
nlp.add_pipe(clean_headlines, before='textcat')

# Parallelize...go!
# with Pool(16) as pool:
#     headlines = pool.map(clean_headlines, df['headline_text'].values)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['headline_text'], 
                                                    df['simple_category'],
                                                    test_size=0.2, train_size=0.8,
                                                    random_state=777, 
                                                    stratify=df['simple_category'])

def generate_label_dict(to_label):
    return {'cats': 
            {'business': to_label == 'business',
             'culture': to_label == 'culture',
             'news': to_label == 'news',
             'opinion': to_label == 'opinion',
             'sport': to_label == 'sport',
             'lifestyle': to_label == 'lifestyle'}}

train_labels = [generate_label_dict(label) for label in y_train]
test_labels = [generate_label_dict(label) for label in y_test]
train = list(zip(X_train, train_labels))

In [ ]:
import random
from spacy.util import minibatch, fix_random_seed, decaying, compounding

# Make it reproducible, yo!
fix_random_seed(777)

optimizer = nlp.begin_training()

losses = {}
dropout = decaying(0.8, 0.2, 1e-4)
batch_sizes = compounding(1, 32, 1.001)

for epoch in range(3):
    random.shuffle(train)
    batches = minibatch(train, size=batch_sizes)

    for batch in batches:
        texts, train_labels = zip(*batch)
        nlp.update(texts, train_labels, drop=next(dropout), sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 678.4694893943288}
{'textcat': 690.3389867024289}


In [ ]:
# Let's save the model so that we can run it again later when we have more memory to spare.
nlp.to_disk('/home/sean/Code/irish_times/bow_3')

In [ ]:
def compare_label(match):
    return 1 if match[0] == match[1] else 0

### Drumroll, please!

In [ ]:
test_test_headlines = [nlp.tokenizer(txt) for txt in X_test]
test_categorizer = nlp.get_pipe('textcat')
test_scores, _ = test_categorizer.predict(test_test_headlines)
test_pred_labels = test_scores.argmax(axis=1)
test_predictions = [test_categorizer.labels[label] for label in test_pred_labels]
compare_test_labels = list(zip(test_predictions, y_test))

# ...aaaaaannnnd the scores!
test_correct = sum([compare_label(m) for m in compare_test_labels])
print(f'The model is {(test_correct / df.shape[0]) * 100:.2f}% accurate on the test data.')

### Additional Notes
It's looking like there are some repeatable things here that would work well in a Hy macro or three.
Consider testing with methods other than `minibatch` as well as `minibatch` using `sklearn.model_selection.cross_val_predict` and train them all at once with the cool multiprocessing thing that I learned.